In [38]:
# =========================
# CELL 1 — Project bootstrap 
# =========================
from pathlib import Path
import os, sys

# Find repo root by walking upward until requirements.txt is found
ROOT = Path().resolve()
while not (ROOT / "requirements.txt").exists() and ROOT != ROOT.parent:
    ROOT = ROOT.parent

if not (ROOT / "requirements.txt").exists():
    raise FileNotFoundError(
        "Could not find requirements.txt. Open this notebook from inside your repo folder."
    )

os.chdir(ROOT)
SRC = ROOT / "src"
if SRC.exists() and str(SRC) not in sys.path:
    sys.path.insert(0, str(SRC))

print("✅ Repo root:", ROOT)
print("✅ Using src path:", SRC if SRC.exists() else "(no src/ found)")
print("✅ CWD:", Path().resolve())


✅ Repo root: /Users/arina/Desktop/market-inefficiencies
✅ Using src path: /Users/arina/Desktop/market-inefficiencies/src
✅ CWD: /Users/arina/Desktop/market-inefficiencies


In [39]:
# =========================
# CELL 2 — Install deps 
# =========================
%pip install -r requirements.txt


Note: you may need to restart the kernel to use updated packages.


In [40]:
# =========================
# CELL 3 — Load config.yaml + decide DB path
# =========================
import yaml
from pathlib import Path

cfg_path = Path("config.yaml")
if not cfg_path.exists():
    raise FileNotFoundError(f"Can't find config.yaml at: {cfg_path.resolve()}")

with cfg_path.open("r") as f:
    cfg = yaml.safe_load(f) or {}


db_path = Path(cfg.get("db_path", "data/raw/dislocations.sqlite3"))
db_path = db_path if db_path.is_absolute() else (Path.cwd() / db_path)
db_path = db_path.resolve()

print("✅ Config:", cfg_path.resolve())
print("✅ DB path:", db_path)
print("✅ DB exists:", db_path.exists())

# Detection parameters (defaults if missing in config)
costs = cfg.get("costs_bps", {}) or {}
fee_bps = float(costs.get("fee_bps", 2))
half_spread_bps = float(costs.get("half_spread_bps", 1))
slippage_bps = float(costs.get("slippage_bps", 3))
threshold_bps = fee_bps + half_spread_bps + slippage_bps

persistence_ms = int(cfg.get("persistence_ms", 300))
lookback_ms = int(cfg.get("lookback_ms", 60*60*1000))  # default: last 60 min

print("✅ threshold_bps:", threshold_bps)
print("✅ persistence_ms:", persistence_ms)
print("✅ lookback_ms:", lookback_ms)


✅ Config: /Users/arina/Desktop/market-inefficiencies/config.yaml
✅ DB path: /Users/arina/Desktop/market-inefficiencies/data/raw/dislocations.sqlite3
✅ DB exists: True
✅ threshold_bps: 6.0
✅ persistence_ms: 300
✅ lookback_ms: 3600000


In [41]:
# =========================
# CELL 4 — Inspect SQLite: list tables + schema + quick sample
# =========================
import sqlite3
import pandas as pd

if not db_path.exists():
    raise FileNotFoundError(f"DB not found at {db_path}")

conn = sqlite3.connect(str(db_path))

tables = pd.read_sql_query(
    "SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;", conn
)
display(tables)

# Expecting a "ticks" table; if not, show what exists and stop
if "ticks" not in set(tables["name"].tolist()):
    conn.close()
    raise RuntimeError("No 'ticks' table found. Check the tables above.")

schema = pd.read_sql_query("PRAGMA table_info(ticks);", conn)
display(schema)

sample = pd.read_sql_query("SELECT * FROM ticks LIMIT 5;", conn)
display(sample)

conn.close()


,name
0,ticks


,cid,name,type,notnull,dflt_value,pk
0,0,ts_ms,INTEGER,1,None,1
1,1,venue,TEXT,1,None,2
2,2,bid,REAL,1,None,0
3,3,ask,REAL,1,None,0
4,4,mid,REAL,1,None,0


,ts_ms,venue,bid,ask,mid
0,1761703795617,COINBASE,112479.48,112479.49,112479.485
1,1761703795659,BITSTAMP,112492.00,112493.00,112492.500
2,1761703795728,KRAKEN,112510.90,112511.00,112510.950
3,1761703795736,BITSTAMP,112492.00,112493.00,112492.500
4,1761703795754,COINBASE,112479.48,112479.49,112479.485


In [42]:
# =========================
# CELL 5 — Make sure output folders exist
# =========================
from pathlib import Path

OUT_TABLES = Path("data/processed")
OUT_FIGS = Path("figures")

OUT_TABLES.mkdir(parents=True, exist_ok=True)
OUT_FIGS.mkdir(parents=True, exist_ok=True)

print("✅ Tables dir:", OUT_TABLES.resolve())
print("✅ Figures dir:", OUT_FIGS.resolve())


✅ Tables dir: /Users/arina/Desktop/market-inefficiencies/data/processed
✅ Figures dir: /Users/arina/Desktop/market-inefficiencies/figures


In [43]:
# =========================
# CELL 6 — Export ALL ticks to CSV (appendix)
# =========================
import sqlite3, pandas as pd

conn = sqlite3.connect(str(db_path))

ticks_all = pd.read_sql_query("SELECT * FROM ticks ORDER BY ts_ms;", conn)
conn.close()

out_csv = OUT_TABLES / "ticks_export.csv"
ticks_all.to_csv(out_csv, index=False)

print("✅ Saved:", out_csv)
print("Rows:", len(ticks_all))
display(ticks_all.head())


✅ Saved: data/processed/ticks_export.csv
Rows: 8039


,ts_ms,venue,bid,ask,mid
0,1761703795617,COINBASE,112479.48,112479.49,112479.485
1,1761703795659,BITSTAMP,112492.00,112493.00,112492.500
2,1761703795728,KRAKEN,112510.90,112511.00,112510.950
3,1761703795736,BITSTAMP,112492.00,112493.00,112492.500
4,1761703795754,COINBASE,112479.48,112479.49,112479.485


In [44]:
# =========================
# CELL 7 — Compute per-venue latest quotes + last-window stats
# =========================
import sqlite3, pandas as pd

conn = sqlite3.connect(str(db_path))

latest_per_venue = pd.read_sql_query(
    """
    WITH latest AS (
      SELECT venue, MAX(ts_ms) AS ts
      FROM ticks
      GROUP BY venue
    )
    SELECT t.venue, t.ts_ms, t.bid, t.ask,
           (t.bid + t.ask)/2.0 AS mid,
           (t.ask - t.bid) AS spread_abs,
           CASE WHEN ((t.bid + t.ask)/2.0) != 0
                THEN 10000.0 * (t.ask - t.bid) / ((t.bid + t.ask)/2.0)
                ELSE NULL
           END AS spread_bps
    FROM ticks t
    JOIN latest l
      ON t.venue = l.venue AND t.ts_ms = l.ts
    ORDER BY t.venue;
    """,
    conn
)
display(latest_per_venue)

window_stats = pd.read_sql_query(
    """
    SELECT
      MIN(ts_ms) AS start_ms,
      MAX(ts_ms) AS end_ms,
      COUNT(*)   AS n_ticks
    FROM ticks
    WHERE ts_ms >= (SELECT MAX(ts_ms) - ? FROM ticks);
    """,
    conn,
    params=(lookback_ms,)
)
display(window_stats)

conn.close()


,venue,ts_ms,bid,ask,mid,spread_abs,spread_bps
0,BITSTAMP,1761704475082,112543.0,112544.00,112543.500,1.00,0.088855
1,COINBASE,1761704474905,112537.5,112537.51,112537.505,0.01,0.000889
2,KRAKEN,1761704475088,112479.6,112479.70,112479.650,0.10,0.008890


,start_ms,end_ms,n_ticks
0,1761703795617,1761704475088,8039


In [45]:
# =========================
# CELL 8 — Build a time series over last lookback window (mid + spread_bps)
# =========================
import sqlite3, pandas as pd

conn = sqlite3.connect(str(db_path))

df = pd.read_sql_query(
    """
    SELECT
      ts_ms,
      venue,
      bid,
      ask,
      (bid + ask)/2.0 AS mid,
      CASE WHEN ((bid + ask)/2.0) != 0
           THEN 10000.0 * (ask - bid) / ((bid + ask)/2.0)
           ELSE NULL
      END AS spread_bps
    FROM ticks
    WHERE ts_ms >= (SELECT MAX(ts_ms) - ? FROM ticks)
    ORDER BY ts_ms;
    """,
    conn,
    params=(lookback_ms,)
)

conn.close()

print("Rows:", len(df))
display(df.head())


Rows: 8039


,ts_ms,venue,bid,ask,mid,spread_bps
0,1761703795617,COINBASE,112479.48,112479.49,112479.485,0.000889
1,1761703795659,BITSTAMP,112492.00,112493.00,112492.500,0.088895
2,1761703795728,KRAKEN,112510.90,112511.00,112510.950,0.008888
3,1761703795736,BITSTAMP,112492.00,112493.00,112492.500,0.088895
4,1761703795754,COINBASE,112479.48,112479.49,112479.485,0.000889


In [46]:
# =========================
# CELL 9 — Detect "events" = spread_bps > threshold_bps for >= persistence_ms
# (done per venue)
# =========================
import numpy as np
import pandas as pd

def detect_events_one_venue(vdf: pd.DataFrame, thr_bps: float, persistence_ms: int) -> pd.DataFrame:
    vdf = vdf.dropna(subset=["spread_bps"]).sort_values("ts_ms").reset_index(drop=True)
    if vdf.empty:
        return pd.DataFrame(columns=["venue","start_ms","end_ms","duration_ms","peak_bps","n_points"])

    above = (vdf["spread_bps"] > thr_bps).to_numpy(dtype=bool)
    ts = vdf["ts_ms"].to_numpy()
    sp = vdf["spread_bps"].to_numpy()

    events = []
    i = 0
    n = len(vdf)
    while i < n:
        if not above[i]:
            i += 1
            continue
        start_i = i
        while i < n and above[i]:
            i += 1
        end_i = i - 1

        start_ms = int(ts[start_i])
        end_ms = int(ts[end_i])
        duration_ms = end_ms - start_ms

        if duration_ms >= persistence_ms:
            peak_bps = float(np.nanmax(sp[start_i:end_i+1]))
            n_points = int(end_i - start_i + 1)
            events.append((start_ms, end_ms, duration_ms, peak_bps, n_points))

    out = pd.DataFrame(events, columns=["start_ms","end_ms","duration_ms","peak_bps","n_points"])
    return out

all_events = []
for venue, vdf in df.groupby("venue"):
    ev = detect_events_one_venue(vdf, threshold_bps, persistence_ms)
    if not ev.empty:
        ev.insert(0, "venue", venue)
        all_events.append(ev)

events = pd.concat(all_events, ignore_index=True) if all_events else pd.DataFrame(
    columns=["venue","start_ms","end_ms","duration_ms","peak_bps","n_points"]
)

print("Events found:", len(events))
display(events.head(20))


Events found: 0


,venue,start_ms,end_ms,duration_ms,peak_bps,n_points


In [ ]:
# =========================
# CELL 10 — Save events tables (raw + pretty UTC)
# =========================
import pandas as pd
import datetime as dt

events_out = OUT_TABLES / "events_last_window.csv"
events.to_csv(events_out, index=False)
print("✅ Saved:", events_out)

def ms_to_utc_str(ms: int) -> str:
    return dt.datetime.utcfromtimestamp(ms / 1000).strftime("%Y-%m-%d %H:%M:%S")

if not events.empty:
    pretty = events.copy()
    pretty["start_utc"] = pretty["start_ms"].apply(ms_to_utc_str)
    pretty["end_utc"] = pretty["end_ms"].apply(ms_to_utc_str)
    pretty["duration_s"] = (pretty["duration_ms"] / 1000.0).round(2)

    pretty_cols = ["venue","start_utc","end_utc","duration_s","peak_bps","n_points"]
    pretty = pretty[pretty_cols].head(50)

    pretty_out = OUT_TABLES / "events_pretty.csv"
    pretty.to_csv(pretty_out, index=False)
    print("✅ Saved:", pretty_out)
    display(pretty)
else:
    print("No events to prettify.")


In [ ]:
# =========================
# CELL 11 — Plots: spread_bps time series + histogram (saved to /figures)
# =========================
import matplotlib.pyplot as plt

# Choose one venue to plot (the busiest, or first one)
venue_counts = df["venue"].value_counts()
plot_venue = venue_counts.index[0] if len(venue_counts) else None

if plot_venue is None:
    raise RuntimeError("No venues in df to plot.")

v = df[df["venue"] == plot_venue].dropna(subset=["spread_bps"]).copy()

# Time series
plt.figure(figsize=(10,4))
plt.plot(v["ts_ms"], v["spread_bps"])
plt.axhline(threshold_bps, linestyle="--")
plt.title(f"Spread (bps) — {plot_venue} — last window")
plt.xlabel("timestamp (ms)")
plt.ylabel("spread_bps")

# shade events (for this venue)
if not events.empty:
    evv = events[events["venue"] == plot_venue]
    for _, r in evv.iterrows():
        plt.axvspan(r["start_ms"], r["end_ms"], alpha=0.15)

ts_out = OUT_FIGS / f"spread_timeseries_{plot_venue}.png"
plt.tight_layout()
plt.savefig(ts_out, dpi=300)
plt.show()
print("✅ Saved:", ts_out)

# Histogram
plt.figure(figsize=(6,4))
plt.hist(v["spread_bps"].dropna(), bins=50)
plt.axvline(threshold_bps, linestyle="--")
plt.title(f"Distribution of spread (bps) — {plot_venue}")
plt.xlabel("spread_bps")
plt.ylabel("count")
hist_out = OUT_FIGS / f"spread_hist_{plot_venue}.png"
plt.tight_layout()
plt.savefig(hist_out, dpi=300)
plt.show()
print("✅ Saved:", hist_out)
